# Creation of a digital twin 
This is a KWH application which allows creating a simple digital twin using python reference implementation. The created digital twin can be launched and connect to S³I in this notebook, until you shut down the kernel. In this notebook, we will borrow the example of harvester to show you how to create a digital twin. All necessary modules are imported into the script, including the S³I library and reference implementation library. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import s3i
import jwt
import requests
import getpass
import json 
from ml.tools import load_config, make_config_file, make_sub_thing
from ml.dt_factory import create_dt_ref
from ml.app_logger import APP_LOGGER, setup_logger
from tools import print_with_timestamp, check_message_encryption, yes, no, dt_creation_app_id, dt_creation_app_secret

## Configure the notebook
In order to use the S³I this notebook needs a client id and the respective secret. Those are already prepared for you. You need to enter your username and password in the following input fields in order to assign this notebook to your personal KWH application. 

In [ ]:
print_with_timestamp("KWH application to create a dt, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=dt_creation_app_id,
                                             client_secret=dt_creation_app_secret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token'''
parsed_username = jwt.decode(access_token, verify=False)["preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

## Create an identity for your digital twin
your digital twin needs also an identify, which can be created using config rest api. 

In [ ]:
create_thing_resp = requests.post(url="https://config.s3i.vswf.dev/things/",
                                  headers={"Authorization": "Bearer {}".format(access_token),
                                           "Content-Type": "application/json"},
                                  data=json.dumps({}))
dt_id = create_thing_resp.json().get("identifier", None)
dt_secret = create_thing_resp.json().get("secret", None)
create_cloud_copy_resp = requests.post(url="https://config.s3i.vswf.dev/things/{}/repository".format(dt_id),
                                       headers={"Authorization": "Bearer {}".format(access_token),
                                                "Content-Type": "application/json"})
create_endpoint_resp = requests.post(url="https://config.s3i.vswf.dev/things/{}/broker".format(dt_id),
                                     headers={"Authorization": "Bearer {}".format(access_token),
                                              "Content-Type": "application/json"},
                                     data=json.dumps({"encrypted": False})
                                     )
print_with_timestamp("Your digital twin has the identifier: {}".format(dt_id))

## Configurate your digital twin
as next we will introduce how to configurate your digital twin Harvester using python reference implementation library. 

In [ ]:
dt_name = input("Please name your digital twin Harvester(e.g. my_harvester): ")

setup_logger(dt_name)
config_engine = make_sub_thing(name="my_engine", roles=["ml40::Engine"],
                               features=["ml40::RotationalSpeed"])
config_cran = make_sub_thing(name="my_bord_computer", roles=["ml40::MachineUI"])
config_file_name = make_config_file(dt_id=dt_id, name=dt_name, roles=["fml40::Harvester"],
                                    features=["fml40::ProvidesProductionData", "fml40::AcceptsFellingJobs",
                                              {"ml40::Composite": [config_engine, config_cran]}])
dt_model = load_config('configs/{}'.format(config_file_name))
print_with_timestamp("The date model of the created harvester: {}" + json.dumps(dt_model, indent=2))

## Create the proxy of your digital twin Harvester

In [ ]:
dt_ref = create_dt_ref(model=dt_model, grant_type="password", secret=dt_secret, username=username, password=password,
                       is_broker_rest=True,
                       is_broker=True, is_repo=False)
dt_proxy = dt_ref.proxy()

## Run the digital twin Harvester

In [ ]:
dt_proxy.run_forever()